In [2]:
import matplotlib.pyplot as plt

from MZPackage.DataPreprocessor import DataPreprocessor
from MZPackage.Utilities import *

In [3]:
dp = DataPreprocessor()
featuresList = ["Iprobe1_a","Iprobe1_b","Iprobe1_c","Iprobe2_a","Iprobe2_b","Iprobe2_c"]

In [4]:
# get benign data and save 
benignData = dp.getNoFaultData(featuresList, datasetFolderName="original_mat")
normalizedBenignData = dp.normalizeDataSetUsingSavedParams(benignData.copy())
saveData(normalizedBenignData, "Anomaly_Detection_Model_Data/normalized_no_fault_data_set")

100%|██████████| 216/216 [00:23<00:00,  9.09it/s]


In [5]:
# generate FDI attack data and save
FDIAttackData = dp.getRandomFDIAttackData([0,1,2],1000, "Anomaly_Detection_Model_Data/normalized_no_fault_data_set")
saveData(FDIAttackData, "Anomaly_Detection_Model_Data/normalized_fdi_attack_data_set")

100%|██████████| 1000/1000 [00:00<00:00, 1495.89it/s]


In [6]:
replayAttackData = dp.getReplayAttackData(1000, "Anomaly_Detection_Model_Data/normalized_no_fault_data_set", "Anomaly_Detection_Model_Data/normalized_data_set")
saveData(replayAttackData, "Anomaly_Detection_Model_Data/normalized_replay_attack_data_set")

In [7]:
featuresToShift = featuresList[0:3]
shiftAttackData = dp.getShiftAttackData(featuresList, featuresToShift, 1000, datasetFolderName = "original_mat")
saveData(shiftAttackData, "Anomaly_Detection_Model_Data/normalized_time_sync_attack_data_set")

100%|██████████| 216/216 [00:23<00:00,  9.16it/s]
